In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize
from numpy.random import rand
f_f = pd.read_csv("./F-F_Research_Data_Factors_daily.CSV")
rtn = pd.read_csv("./DailyReturn.csv")


In [2]:
#OLS calculate beta
stocks= ['AAPL', 'FB', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']
rtn = rtn[stocks]
cov = rtn.cov()


def beta_calc(ff, d_rtn, param, rf, rs):
    x = np.array(ff[param].tolist())[-60:]
    y1 = np.array(ff[rf].tolist())[-60:]
    y2 = np.array(d_rtn[rs].tolist())
    y = y2 - y1
    # adding the constant term
    x = sm.add_constant(x)

    result = sm.OLS(y, x).fit()
    return result.params[1]
stock_rtn = pd.DataFrame(columns = stocks)
row = []
for stock in stocks:
    beta_mkt = beta_calc(f_f, rtn, "Mkt-RF", "RF", stock)
    beta_smb = beta_calc(f_f, rtn, "SMB", "RF", stock)
    beta_hml = beta_calc(f_f, rtn, "HML", "RF", stock)
    exp_rs = f_f["RF"].mean() + beta_mkt*(f_f["Mkt-RF"].mean()) + beta_smb*(f_f["SMB"].mean()) + beta_hml*(f_f["HML"].mean())
    row.append(exp_rs)
stock_rtn.loc[len(stock_rtn.index)] = row
stock_rtn

,AAPL,FB,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
0,0.012163,0.012056,0.012147,0.012016,0.012136,0.011982,0.01206,0.012236,0.012054,0.012146,0.01216,0.012022,0.012061,0.012072,0.01205,0.01212,0.01209,0.012162,0.012051,0.012196


In [3]:
stock_rtn = np.array(stock_rtn)

In [4]:
#construct covariance matrix

In [5]:
def objective(w):
    global cov
    return np.matmul(np.matmul(w,np.array(cov)),np.transpose(w))

In [6]:

cons = ({'type':'eq', 'fun': lambda w: sum(w) - 1},
        {'type':'eq', 'fun': lambda w: (w*stock_rtn).sum() - 0.04}
        )
# perform the l-bfgs-b algorithm search
w = [0.05]*20
bounds = ((0, 1) for i in range(0,20))
result = minimize(objective, w, method = "trust-constr", bounds = bounds, constraints = cons)
# summarize the result
# print(result)


/Users/staceywzy/PycharmProjects/FinTech590-RiskManagement/venv/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/Users/staceywzy/PycharmProjects/FinTech590-RiskManagement/venv/lib/python3.10/site-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  warn('Singular Jacobian matrix. Using SVD decomposition to ' +


In [7]:
result.x

array([ 0.75618169, -0.24243637,  0.75618129, -0.24243734, -0.24243439,
       -0.24243817, -0.24243628,  0.75618349, -0.24243641, -0.14073608,
        0.7561816 , -0.24243721, -0.24243625, -0.24243598, -0.24243651,
       -0.2424348 , -0.24243555,  0.75618164, -0.24243648,  0.75618248])